# Error Handling Example Notebook

This notebook contains various error scenarios to test the error handling and testing capabilities of NB-QOL.

## Setup

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import traceback

In [ ]:
# Helper function to display errors nicely
def try_execute(func, *args, **kwargs):
    try:
        return func(*args, **kwargs)
    except Exception as e:
        print(f"\033[91mError: {type(e).__name__}: {e}\033[0m")
        return None

## 1. Basic Error Types

In [ ]:
# Syntax Error (uncomment to see)
# this is a syntax error:

In [ ]:
# NameError
try_execute(lambda: undefined_variable)

In [ ]:
# TypeError
try_execute(lambda: "string" + 5)

In [ ]:
# ValueError
try_execute(lambda: int("not a number"))

In [ ]:
# ZeroDivisionError
try_execute(lambda: 1/0)

In [ ]:
# IndexError
my_list = [1, 2, 3]
try_execute(lambda: my_list[10])

In [ ]:
# KeyError
my_dict = {"a": 1, "b": 2}
try_execute(lambda: my_dict["c"])

In [ ]:
# AttributeError
try_execute(lambda: "string".nonexistent_method())

## 2. Library-specific Errors

In [ ]:
# NumPy Error
try_execute(lambda: np.array([1, 2, 3]) / np.array([1, 0, 1]))

In [ ]:
# Pandas Error
try_execute(lambda: pd.DataFrame({"A": [1, 2, 3]}).loc[5])

In [ ]:
# Matplotlib Error
try_execute(lambda: plt.plot(np.array([1, 2, 3]), np.array([1, 2])))

## 3. Custom Exceptions

In [ ]:
# Define custom exceptions
class MyCustomError(Exception):
    """Base class for custom exceptions"""
    pass

class ValueTooLargeError(MyCustomError):
    """Raised when the input value is too large"""
    pass

class ValueTooSmallError(MyCustomError):
    """Raised when the input value is too small"""
    pass

In [ ]:
# Function that raises custom exceptions
def validate_value(value):
    if value > 100:
        raise ValueTooLargeError(f"Value {value} is too large (max 100)")
    elif value < 0:
        raise ValueTooSmallError(f"Value {value} is too small (min 0)")
    return f"Value {value} is valid"

# Test with valid and invalid values
print("Testing with value 50:")
try_execute(validate_value, 50)

print("\nTesting with value 150:")
try_execute(validate_value, 150)

print("\nTesting with value -10:")
try_execute(validate_value, -10)

## 4. Error with Traceback

In [ ]:
# Function with nested calls to generate a deeper traceback
def level1():
    return level2()

def level2():
    return level3()

def level3():
    return level4()

def level4():
    # Generate an error
    return 1 / 0

# Try to execute with full traceback
try:
    level1()
except Exception as e:
    print(f"Error: {type(e).__name__}: {e}")
    traceback.print_exc()

## 5. Catching and Handling Multiple Exceptions

In [ ]:
def process_data(data, index, divisor):
    try:
        # Could raise TypeError if data is not subscriptable
        value = data[index]
        
        # Could raise ZeroDivisionError
        result = value / divisor
        
        # Could raise ValueError if value is not convertible to int
        int_result = int(result)
        
        return int_result
    
    except (IndexError, KeyError) as e:
        print(f"Access error: {e}")
        return None
        
    except ZeroDivisionError:
        print("Cannot divide by zero")
        return None
        
    except ValueError as e:
        print(f"Conversion error: {e}")
        return None
        
    except Exception as e:
        print(f"Unexpected error: {type(e).__name__}: {e}")
        return None

In [ ]:
# Test various error scenarios
test_cases = [
    {"name": "Valid case", "data": [10, 20, 30], "index": 1, "divisor": 2},
    {"name": "IndexError", "data": [10, 20, 30], "index": 5, "divisor": 2},
    {"name": "ZeroDivisionError", "data": [10, 20, 30], "index": 1, "divisor": 0},
    {"name": "KeyError", "data": {"a": 10, "b": 20}, "index": "c", "divisor": 2},
    {"name": "TypeError", "data": 10, "index": 0, "divisor": 2},
    {"name": "ValueError", "data": ["10.5", "20.5", "30.5"], "index": 1, "divisor": 2},
]

for case in test_cases:
    print(f"\n{case['name']}:")
    try:
        result = process_data(case['data'], case['index'], case['divisor'])
        print(f"Result: {result}")
    except Exception as e:
        print(f"Unexpected exception: {type(e).__name__}: {e}")

## 6. File-related Errors

In [ ]:
# FileNotFoundError
try_execute(lambda: open("nonexistent_file.txt", "r"))

In [ ]:
# PermissionError (might not trigger depending on environment)
try:
    # Create a file
    with open("test_file.txt", "w") as f:
        f.write("Test content")
    
    # Try to change permissions (this might fail in some environments)
    try:
        os.chmod("test_file.txt", 0o000)  # Remove all permissions
    except Exception as e:
        print(f"Could not change permissions: {e}")
        
    # Try to write to the file
    with open("test_file.txt", "w") as f:
        f.write("More content")
except Exception as e:
    print(f"Error: {type(e).__name__}: {e}")
finally:
    # Clean up
    try:
        os.chmod("test_file.txt", 0o666)  # Restore permissions
        os.remove("test_file.txt")
    except:
        pass

## 7. Context Manager Errors

In [ ]:
# Custom context manager that can generate errors
class ErrorInContext:
    def __init__(self, raise_on_enter=False, raise_on_exit=False):
        self.raise_on_enter = raise_on_enter
        self.raise_on_exit = raise_on_exit
    
    def __enter__(self):
        print("Entering context")
        if self.raise_on_enter:
            raise ValueError("Error during __enter__")
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        print("Exiting context")
        if self.raise_on_exit:
            raise RuntimeError("Error during __exit__")
        # Return True to suppress any exception that occurred in the with block
        return False  # Don't suppress exceptions

In [ ]:
# Test context manager with different error scenarios
scenarios = [
    {"name": "No errors", "raise_on_enter": False, "raise_on_exit": False, "raise_in_block": False},
    {"name": "Error on enter", "raise_on_enter": True, "raise_on_exit": False, "raise_in_block": False},
    {"name": "Error in block", "raise_on_enter": False, "raise_on_exit": False, "raise_in_block": True},
    {"name": "Error on exit", "raise_on_enter": False, "raise_on_exit": True, "raise_in_block": False},
    {"name": "Error in block and exit", "raise_on_enter": False, "raise_on_exit": True, "raise_in_block": True},
]

for scenario in scenarios:
    print(f"\n{scenario['name']}:")
    try:
        with ErrorInContext(scenario['raise_on_enter'], scenario['raise_on_exit']) as ctx:
            print("Inside context block")
            if scenario['raise_in_block']:
                raise ValueError("Error inside context block")
            print("Context block completed successfully")
        print("After context manager")
    except Exception as e:
        print(f"Caught exception: {type(e).__name__}: {e}")

## 8. Asyncio Errors

In [ ]:
# Asyncio errors
import asyncio

async def async_function_with_error():
    await asyncio.sleep(0.1)
    raise ValueError("Async error occurred")

async def main():
    try:
        await async_function_with_error()
    except Exception as e:
        print(f"Caught async exception: {type(e).__name__}: {e}")

# Run the async function
try:
    asyncio.run(main())
except Exception as e:
    print(f"Uncaught exception: {type(e).__name__}: {e}")

## 9. Memory Error Simulation

In [ ]:
# Simulate memory error (don't uncomment unless you want to potentially crash the kernel)
def simulate_memory_error():
    # This will attempt to allocate an excessively large array
    # Uncomment at your own risk!
    # big_array = [0] * (10**9)  # 1 billion elements
    print("Memory allocation would happen here (commented out for safety)")

try_execute(simulate_memory_error)

## 10. Error Recovery and Cleanup

In [ ]:
# Demonstrate try-except-else-finally pattern
def process_with_cleanup(items, index):
    resources = []
    try:
        print(f"Processing item at index {index}")
        
        # Allocate some "resources"
        resources.append("Resource 1")
        print("Allocated Resource 1")
        
        # This could fail
        item = items[index]
        
        # Allocate more resources after the risky operation
        resources.append("Resource 2")
        print("Allocated Resource 2")
        
        # Another operation that could fail
        result = 100 / item
        
    except IndexError as e:
        print(f"Index error during processing: {e}")
        return None
        
    except ZeroDivisionError as e:
        print(f"Division error during processing: {e}")
        return None
        
    except Exception as e:
        print(f"Unexpected error during processing: {type(e).__name__}: {e}")
        return None
        
    else:
        # This only executes if no exception was raised
        print(f"Processing succeeded with result: {result}")
        return result
        
    finally:
        # This always executes, regardless of whether an exception was raised
        print(f"Cleaning up {len(resources)} resources...")
        for resource in resources:
            print(f"Released {resource}")
        print("Cleanup complete")

In [ ]:
# Test successful case
print("Case 1: Successful processing\n")
process_with_cleanup([10, 20, 30], 1)

# Test index error
print("\nCase 2: Index error\n")
process_with_cleanup([10, 20, 30], 5)

# Test division by zero
print("\nCase 3: Division by zero\n")
process_with_cleanup([10, 0, 30], 1)

## Conclusion

This notebook demonstrates various error scenarios that can be useful for testing the error handling and testing capabilities of NB-QOL. It includes:

1. Basic Python error types
2. Library-specific errors
3. Custom exceptions
4. Errors with traceback information
5. Handling multiple exception types
6. File-related errors
7. Context manager errors
8. Asyncio errors
9. Memory error simulation
10. Error recovery and cleanup patterns

These examples provide a comprehensive test suite for how NB-QOL handles error conditions when executing, testing, or converting notebooks with errors.